# IMPORTING NEEDED PACKAGES

In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


### FOURSQUARE CREDENTIALS

In [2]:
CLIENT_ID = 'UWBVYBR3EUAZUIUXONHBKCMPP51UFA2NZ4EZCGPVL3K5P02E' # your Foursquare ID
CLIENT_SECRET = '2EP3XVUZ2N2UTVMFSM0VVJO2ZO5IX5MCMTPCNW1L05FAG5AD' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('My Credentails:')
print('CLIENT_ID: ' + CLIENT_ID)

My Credentails:
CLIENT_ID: UWBVYBR3EUAZUIUXONHBKCMPP51UFA2NZ4EZCGPVL3K5P02E


## Creating the GET request URL

In [3]:
# type your answer here
LIMIT = 500 # Maximum is 100
cities = ["New York, NY", 'San Francisco, CA']
results = {}
for city in cities:
    url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&near={}&limit={}&categoryId={}'.format(
        CLIENT_ID, 
        CLIENT_SECRET, 
        VERSION, 
        city,
        LIMIT,
        "4bf58dd8d48988d1ca941735") # PIZZA PLACE CATEGORY ID
    results[city] = requests.get(url).json()

## The Data

In [4]:
df_venues={}
for city in cities:
    venues = json_normalize(results[city]['response']['groups'][0]['items'])
    df_venues[city] = venues[['venue.name', 'venue.location.address', 'venue.location.lat', 'venue.location.lng']]
    df_venues[city].columns = ['Name', 'Address', 'Lat', 'Lng']

/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/ipykernel_launcher.py:3: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  This is separate from the ipykernel package so we can avoid doing imports until


In [27]:
df_venues

{'New York, NY':                                    Name                      Address  \
 0                                B Side                370 W 51st St   
 1                   Prince Street Pizza                 27 Prince St   
 2                           Joe's Pizza                150 E 14th St   
 3                    Emily West Village                35 Downing St   
 4                           Joe's Pizza                 7 Carmine St   
 5                                 Posto                  310 2nd Ave   
 6         San Matteo Pizza Espresso Bar                 1739 2nd Ave   
 7                        Champion Pizza              17 Cleveland Pl   
 8                         Scarr's Pizza                22 Orchard St   
 9                           Joe's Pizza                1435 Broadway   
 10            John's of Bleecker Street              278 Bleecker St   
 11            Garlic New York Pizza Bar                  629 2nd Ave   
 12               Vezzo Thin Crust 

The Foursquare API Only gives us the nearest 100 venues in the city.

## Visualizing the Densities

In [5]:
maps = {}
for city in cities:
    city_lat = np.mean([results[city]['response']['geocode']['geometry']['bounds']['ne']['lat'],
                        results[city]['response']['geocode']['geometry']['bounds']['sw']['lat']])
    city_lng = np.mean([results[city]['response']['geocode']['geometry']['bounds']['ne']['lng'],
                        results[city]['response']['geocode']['geometry']['bounds']['sw']['lng']])
    maps[city] = folium.Map(location=[city_lat, city_lng], zoom_start=11)

    # add markers to map
    for lat, lng, label in zip(df_venues[city]['Lat'], df_venues[city]['Lng'], df_venues[city]['Name']):
        label = folium.Popup(label, parse_html=True)
        folium.CircleMarker(
            [lat, lng],
            radius=5,
            popup=label,
            color='blue',
            fill=True,
            fill_color='#3186cc',
            fill_opacity=0.7,
            parse_html=False).add_to(maps[city])  
    print(f"Total number of pizza places in {city} = ", results[city]['response']['totalResults'])
    
    print("Showing Top 100")
    print("_________________________________________________________")

Total number of pizza places in New York, NY =  271
Showing Top 100
_________________________________________________________
Total number of pizza places in San Francisco, CA =  167
Showing Top 100
_________________________________________________________


In [6]:
maps[cities[0]]

In [9]:
maps[cities[1]]

It can be seen that both New York and San Francisco are dense cities with Pizza places.  

However, Let's have a concrete measure of this density. 

 

## Measuing density of the two cities .

Basic statistics will be used for this: 
1. Working out the mean location of the pizza places which should be near to most of them if they are really dense or far if not. 

2. Calculating the average of the distance of the venues to the mean coordinates.

In [7]:
maps = {}
for city in cities:
    city_lat = np.mean([results[city]['response']['geocode']['geometry']['bounds']['ne']['lat'],
                        results[city]['response']['geocode']['geometry']['bounds']['sw']['lat']])
    city_lng = np.mean([results[city]['response']['geocode']['geometry']['bounds']['ne']['lng'],
                        results[city]['response']['geocode']['geometry']['bounds']['sw']['lng']])
    maps[city] = folium.Map(location=[city_lat, city_lng], zoom_start=11)
    venues_mean_coor = [df_venues[city]['Lat'].mean(), df_venues[city]['Lng'].mean()] 
    # add markers to map
    for lat, lng, label in zip(df_venues[city]['Lat'], df_venues[city]['Lng'], df_venues[city]['Name']):
        label = folium.Popup(label, parse_html=True)
        folium.CircleMarker(
            [lat, lng],
            radius=5,
            popup=label,
            color='blue',
            fill=True,
            fill_color='#3186cc',
            fill_opacity=0.7,
            parse_html=False).add_to(maps[city])
        folium.PolyLine([venues_mean_coor, [lat, lng]], color="green", weight=1.5, opacity=0.5).add_to(maps[city])
    
    label = folium.Popup("Mean Co-ordinate", parse_html=True)
    folium.CircleMarker(
        venues_mean_coor,
        radius=10,
        popup=label,
        color='green',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(maps[city])

    print(city)
    print("Mean Distance from Mean coordinates")
       
    print(np.mean(np.apply_along_axis(lambda x: np.linalg.norm(x - venues_mean_coor),1,df_venues[city][['Lat','Lng']].values)))
    print("___________________________________")

New York, NY
Mean Distance from Mean coordinates
0.02296898464975578
___________________________________
San Francisco, CA
Mean Distance from Mean coordinates
0.02880416941977326
___________________________________


In [25]:
maps[cities[0]]

In [26]:
maps[cities[1]]

### It can be seen that New York is the best option. 

### The only remamining now is to go out and start eating from the various eating places identified.